In [15]:
!pip install keras
!pip install plotly
!pip install cufflinks

  Created wheel for retrying: filename=retrying-1.3.3-cp36-none-any.whl size=11435 sha256=2600a511554f244b4b53c547dd38b6d56618c26ca409c3d36f392a61691ee261
  Stored in directory: C:\Users\Ishan\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
  Created wheel for cufflinks: filename=cufflinks-0.17.0-cp36-none-any.whl size=68553 sha256=a2db332c31bd4c0e4007fa0c9ddf5e05e3919025f97e8b385605eb4071dd160a
  Stored in directory: C:\Users\Ishan\AppData\Local\pip\Cache\wheels\44\d7\dc\e830ab00bc2dd3b2731295103baa070f8cbdda8891f71a7a8d
Successfully built cufflinks


In [17]:
!pip install chart_studio

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
#import chart_studio.plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
#import chart_studio.plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
#from chart_studio.plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('df_3mil.csv', index_col=False )
df.head()
df.drop(columns = 'Unnamed: 0', inplace = True)

,Unnamed: 0,type,title,label,tokens,lemma
0,0,fake,Veteran Commentator Calls Out the Growing “Eth...,1,"['Veteran', 'Commentator', 'Calls', 'Out', 'th...",Veteran Commentator Calls Out the Growing Ethn...
1,1,fake,Scientists move Doomsday Clock ahead by 30 sec...,1,"['Scientists', 'move', 'Doomsday', 'Clock', 'a...",Scientists move Doomsday Clock ahead by 30 sec...
2,2,fake,Poll: Calls for War From Israelis and Palestin...,1,"['Poll', 'Calls', 'for', 'War', 'From', 'Israe...",Poll Calls for War From Israelis and Palestini...
3,3,fake,"College Basketball Rankings, Week 12: Top 25 P...",1,"['College', 'Basketball', 'Rankings', 'Week', ...",College Basketball Rankings Week 12 Top 25 Pow...
4,4,fake,Conservative Fighters,1,"['Conservative', 'Fighters']",Conservative Fighters


In [3]:
df.head()

,type,title,label,tokens,lemma
0,fake,Veteran Commentator Calls Out the Growing “Eth...,1,"['Veteran', 'Commentator', 'Calls', 'Out', 'th...",Veteran Commentator Calls Out the Growing Ethn...
1,fake,Scientists move Doomsday Clock ahead by 30 sec...,1,"['Scientists', 'move', 'Doomsday', 'Clock', 'a...",Scientists move Doomsday Clock ahead by 30 sec...
2,fake,Poll: Calls for War From Israelis and Palestin...,1,"['Poll', 'Calls', 'for', 'War', 'From', 'Israe...",Poll Calls for War From Israelis and Palestini...
3,fake,"College Basketball Rankings, Week 12: Top 25 P...",1,"['College', 'Basketball', 'Rankings', 'Week', ...",College Basketball Rankings Week 12 Top 25 Pow...
4,fake,Conservative Fighters,1,"['Conservative', 'Fighters']",Conservative Fighters


In [4]:
for i in df.index:
    try:
        df['lemma'][i] = df['lemma'][i].lower()
    except AttributeError:
        df.drop(index = i, inplace = True)
    
    

KeyboardInterrupt: 

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(df['lemma'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(df['lemma'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
epochs = 5
batch_size = 512

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])